In [4]:
from datetime import datetime, timedelta
import pandas as pd
import plotly.figure_factory as ff
from ortools.sat.python import cp_model
from bisect import bisect_left, bisect_right
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import unittest
from src.objects.timeinterval import *
from src.utils.utils import *
import uuid

In [42]:
intervals = TimeIntervals([TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 11)),
                           TimeInterval(datetime(2023, 2, 1, 12), datetime(2023, 2, 1, 13)),
                           TimeInterval(datetime(2023, 2, 1, 14), datetime(2023, 2, 1, 16))])
intervals.visualize_gantt()


In [84]:
# Create employee class, which contains the employee's name, 
# and the time intervals that the employee is available for given periods of time
# assigned shifts will be stored in a list, which will be updated as the schedule is created
# and the employee's availability will be updated as shifts are assigned
class Employee:
    def __init__(self, name: str, age: int, gender: str,  availability: TimeIntervals):
        self.name = name
        self.age = age
        self.gender = gender
        self.availability = availability
        self.assigned_shifts = []
    
    # add a shift to the employee's assigned shifts
    def add_shift(self, shift):
        self.assigned_shifts.append(shift)
    
    # remove a shift from the employee's assigned shifts
    def remove_shift(self, shift):
        self.assigned_shifts.remove(shift)
        

    


In [103]:
# Create Shift class, which contains the shift's name,
# the time interval that the shift takes place,
# the number of employees that are needed for the shift,
# the assigned employees to the shift,
# and the number of employees that are currently assigned to the shift, which is initially 0
# the workload coefficient of the shift, 
# shift_type is a string that can be anything, but it is used to differentiate and group shifts together
# The shift's name is optional, and if it is not provided, it will be set to the shift's time interval
# The number of employees that are needed for the shift is optional, and if it is not provided, it will be set to 1
# The workload coefficient of the shift is optional, and if it is not provided, it will be set to 1

class Shift:
    def __init__(self, interval: TimeInterval, name: str = "New shift", min_employees: int = 1, max_employees: int = 1, shift_name: str = None, shift_type: str = "Uncategorized", workload_coefficient: int = 1):
        self.name = name
        assert min_employees <= max_employees, "Minimum number of employees cannot be greater than maximum number of employees"
        self.interval = interval
        self.min_employees = min_employees
        self.max_employees = max_employees
        self.assigned_employees = []
        self.num_assigned_employees = 0
        self.shift_name = shift_name
        self.shift_type = shift_type
        self.workload_coefficient = workload_coefficient
        self.start_time = self.interval.start_time
        self.end_time = self.interval.end_time
        self.duration = self.interval.duration()

        if self.shift_name is None:
            self.shift_name = self.interval.get_start_time().strftime("%H:%M") + " - " + self.interval.get_end_time().strftime("%H:%M")

    def __str__(self):
        return self.shift_name

    def __repr__(self):
        return str(self)

    def add_assigned_employee(self, employee: Employee):
        # check if the employee is already assigned to the shift
        if employee in self.assigned_employees:
            raise Exception("Employee is already assigned to this shift")
        # check if the shift is full
        if self.num_assigned_employees == self.max_employees:
            raise Exception("Shift is full")

        self.assigned_employees.append(employee)
        self.num_assigned_employees += 1

        # update the employee's availability, assigned shifts
        employee.availability = employee.availability - self.interval
        employee.add_shift(self)

    def remove_assigned_employee(self, employee: Employee):
        # check if the employee is assigned to the shift
        if employee not in self.assigned_employees:
            raise Exception("Employee is not assigned to this shift")

        self.assigned_employees.remove(employee)
        self.num_assigned_employees -= 1

        # update the employee's availability, assigned shifts
        employee.availability = employee.availability + self.interval
        employee.remove_shift(self)
    
    def reset_employees(self):
        for employee in self.assigned_employees:
            employee.availability = employee.availability + self.interval
            employee.remove_shift(self)

        self.assigned_employees = []
        self.num_assigned_employees = 0

    def change_shift_type(self, shift_type: str):
        self.shift_type = shift_type

    def change_workload_coefficient(self, workload_coefficient: int):
        self.workload_coefficient = workload_coefficient

    def change_min_employees(self, min_employees: int):
        self.min_employees = min_employees

    def change_max_employees(self, max_employees: int):
        self.max_employees = max_employees

    def change_shift_name(self, shift_name: str):
        self.shift_name = shift_name

    def change_interval(self, interval: TimeInterval):
        self.interval = interval
        self.start_time = self.interval.start_time
        self.end_time = self.interval.end_time
        self.duration = self.interval.duration()


# RecurringShift class, inherits from Shift class and adds a recurrence interval
# The recurrence interval is a Timedelta object that represents the interval between each recurrence of the shift
# The recurrence interval is mandatory
class RecurringShift(Shift):
    def __init__(self, interval: TimeInterval, recurrence_interval: timedelta, name: str = "New shift", min_employees: int = 1, max_employees: int = 1, shift_name: str = None, shift_type: str = "Uncategorized", workload_coefficient: int = 1):
        super().__init__(interval, name, min_employees, max_employees, shift_name, shift_type, workload_coefficient)
        self.recurrence_interval = recurrence_interval

    def __str__(self):
        return self.shift_name

    def __repr__(self):
        return str(self)


    # def get_recurrence_intervals(self, start_date: datetime, end_date: datetime):
    #     # get the start and end time of the shift
    #     start_time = self.interval.start_time
    #     end_time = self.interval.end_time

    #     # get the start and end date of the recurrence interval
    #     recurrence_start_date = start_date
    #     recurrence_end_date = end_date

    #     # get the start and end time of the recurrence interval
    #     recurrence_start_time = recurrence_start_date + start_time - start_date
    #     recurrence_end_time = recurrence_end_date + end_time - end_date

    #     # get the recurrence interval
    #     recurrence_interval = self.recurrence_interval

    #     # get the recurrence intervals
    #     recurrence_intervals = TimeIntervals()
    #     recurrence_intervals.add_interval(TimeInterval(recurrence_start_time, recurrence_end_time))
    #     while recurrence_end_time < end_date:
    #         recurrence_start_time += recurrence_interval
    #         recurrence_end_time += recurrence_interval
    #         recurrence_intervals.add_interval(TimeInterval(recurrence_start_time, recurrence_end_time))

    #     return recurrence_intervals

    # def get_recurrence_shifts(self, start_date: datetime, end_date: datetime):
    #     # get the recurrence intervals
    #     recurrence_intervals = self.get_recurrence_intervals(start_date, end_date)

    #     # get the recurrence shifts
    #     recurrence_shifts = []
    #     for recurrence_interval in recurrence_intervals:
    #         recurrence_shift = Shift(recurrence_interval, self.name, self.min_employees, self.max_employees, self.shift_name, self.shift_type, self.workload_coefficient)
    #         recurrence_shifts.append(recurrence_shift)

    #     return recurrence_shifts

    # def change_recurrence_interval(self, recurrence_interval: timedelta):
    #     self.recurrence_interval = recurrence_interval




In [38]:
s1 = Shift(TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 11)), shift_name="Shift 1")

In [108]:
# Create Schedule class, which contains the schedule's name,
# the list of employees, the list of shifts, and the list of shift types that are in the schedule
# the schedule's name is optional, and if it is not provided, it will be set to 'New Schedule'
class Schedule:
    def __init__(self, name: str = 'New Schedule'):
        self.name = name
        self.employees = []
        self.shifts = []
        self.shift_types = []
        self.start_time = None
        self.end_time = None
   
    # add an employee to the schedule
    def add_employee(self, name: str = None, gender: str = None, availability: TimeInterval = None):
        # if the employee's name is not provided, set it to 'Employee {len(self.employees) + 1}'
        if name is None:
            name = "Employee " + str(len(self.employees) + 1)
        
        # if availability is not provided, set it to the entire schedule
        if (availability is None) and (self.start_time is not None) and (self.end_time is not None):
            availability = TimeInterval(self.start_time, self.end_time)
        
        # create the employee
        employee = Employee(name = name, age = None, gender = None, availability = availability)
        self.employees.append(employee)

        return employee

    def add_shift(self, interval: TimeInterval, name: str = None, shift_name: str = None, shift_type: str = "Uncategorized", min_employees: int = 1, max_employees: int = 1, workload_coefficient: int = 1, recurring: bool = False, recurring_interval: timedelta = None, end_date: datetime = None):
        # if the shift's name is not provided, set it to 'Shift {len(self.shifts) + 1}'
        if name is None:
            name = "Shift " + str(len(self.shifts) + 1)        
    
        # update the schedule's start and end times if the shift's interval is outside of the schedule's interval
        if self.start_time is None:
            self.start_time = interval.start_time
        elif interval.start_time < self.start_time:
            self.start_time = interval.start_time
        if self.end_time is None:
            self.end_time = interval.end_time
        elif interval.end_time > self.end_time:
            self.end_time = interval.end_time

        # create the shift, single or recurring
        if recurring:
            if recurring_interval is None:
                raise Exception("Recurring interval must be provided if shift is recurring")
            shifts = []
            for i in range(0, (end_date - interval.start_time).days, recurring_interval.days):
                shift = Shift(interval + timedelta(days = i), name, shift_name, shift_type, min_employees, max_employees, workload_coefficient)
                shifts.append(shift)
        else:
            shift = Shift(interval, name, shift_name, shift_type, min_employees, max_employees, workload_coefficient)
            shifts = [shift]

        # add the shift to the list of shifts
        self.shifts.extend(shifts)
                

        # add the shift type to the list of shift types if it is not already in the list
        if shift_type not in self.shift_types:
            self.shift_types.append(shift_type)

        return shifts
    

    # reset the schedule
    def reset(self):
        self.employees = []
        self.shifts = []
        self.shift_types = []

    # change the schedule's name
    def change_name(self, name: str):
        self.name = name

    # show the schedule in a format that is easy to read (shift name, type, assigned employees)
    def show(self):
        print(self.name)
        print("Start time:", self.start_time)
        print("End time:", self.end_time)
        for shift in self.shifts:
            print(shift.shift_name, shift.shift_type, shift.assigned_employees)




    


In [109]:
# Test the schedule
schedule = Schedule('Test Schedule')

# Add employees
employee1 = schedule.add_employee()
employee1.name = "Patipan"
employee1.availability = TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 11))
employee2 = schedule.add_employee()
employee2.name = "Pattaraporn"


# check employee  in schedule
print([employee.name for employee in schedule.employees])

# Add shifts
shift1 = schedule.add_shift(TimeInterval(datetime(2023, 2, 1, 10), datetime(2023, 2, 1, 11)), shift_name="Shift 1")

# check shift in schedule
print([shift.name for shift in schedule.shifts])



['Patipan', 'Pattaraporn']
['Shift 1']
